In [33]:
%env APP_DIR=/data/zerospeech/zrbench
from pathlib import Path
from zerospeech.benchmarks.sLM_21 import SLM21Submission
from zerospeech.benchmarks.sLM_21.validators import SLM21SubmissionValidator

submission = SLM21Submission.load(Path('/data/zerospeech/zrbench/samples/sLM21-random-submission'))

validator = SLM21SubmissionValidator()

res = validator.validate(submission)

env: APP_DIR=/data/zerospeech/zrbench


In [34]:
%env APP_DIR=/data/zerospeech/zrbench
from zerospeech.benchmarks.validation import show_errors

for r in res:
    print(r)
#
# print('-'*20)
# print(f"{str(res[1])} -> {len(res[1].data)}")
# print(len(validator.dataset.index.subsets.lexical_dev.items.wav_list.files_list))
# print('-'*20)
# # print(res[1].data)

env: APP_DIR=/data/zerospeech/zrbench
ValidationOK(lexical_dev): dataframe validates tests
ValidationOK(lexical_dev): expected files found
ValidationOK(lexical_test): dataframe validates tests
ValidationOK(lexical_test): expected files found
